In [1]:
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from sys import stdout
import os
import time

pdb = PDBFile('NF1-SPRED_D1217G-processed.pdb')
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
system = forcefield.createSystem(pdb.topology, nonbondedMethod=PME,
        nonbondedCutoff=1*nanometer, constraints=HBonds)

PDBFile.writeFile(pdb.topology, pdb.positions, open('NF1-SPRED_D1217G-opemm.pdb','w'))

checkpoint_file = 'checkpoint.cpt'
overwrite = True
replica = 1

In [2]:
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
simulation = Simulation(pdb.topology, system, integrator)

if os.path.exists(checkpoint_file) and not overwrite:
    with open(checkpoint_file, 'rb') as cpt:
        simulation.context.loadCheckpoint(cpt.read())
        
else:
    simulation.context.setPositions(pdb.positions)    
    simulation.minimizeEnergy()
    
simulation.reporters.append(DCDReporter('output_NF1-SPRED_D1217G_'+str(replica).zfill(2)+'.dcd', 100))
simulation.reporters.append(StateDataReporter(stdout, 10000, step=True,
        potentialEnergy=True, temperature=True))
simulation.reporters.append(CheckpointReporter(checkpoint_file, 100))
start_time = time.time()
simulation.step(300000)
print('It ran for %s s' % (time.time() - start_time))


#"Step","Potential Energy (kJ/mole)","Temperature (K)"
10000,-1580933.3331094359,300.4639768254012
20000,-1578581.6476784921,300.6260605273652
30000,-1579770.8842847922,300.08147770619166
40000,-1582004.9201688361,299.86231999776413
50000,-1582099.9843595335,298.1648072449374
60000,-1582063.7584004218,301.66009529105224
70000,-1581344.9542537506,298.9626692182171
80000,-1581343.2750216287,300.9411049725288
90000,-1582358.7875311978,298.7509532274964
100000,-1582194.1198111705,301.72687277838384
110000,-1581910.6595774947,300.84893578901784
120000,-1582223.7319677412,299.319866636512
130000,-1583716.4469778526,296.71260023628315
140000,-1579879.2350872129,299.9287713815859
150000,-1580841.7540857478,298.52486398866023
160000,-1582147.160865125,300.41202511693524
170000,-1583686.559370059,299.4063372880296
180000,-1583088.1785477302,301.117928830778
190000,-1583121.4215649809,299.47601845020773
200000,-1581381.0071028285,299.6550745015734
210000,-1582904.2785504558,300.34943253443566
220

In [5]:
import mdtraj as md
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
traj = md.load('output_NF1-SPRED_D1217G_'+str(replica).zfill(2)+'.dcd', top='NF1-SPRED_D1217G-opemm.pdb')
traj.image_molecules(inplace=True)
traj.save('output_NF1-SPRED_'+str(replica).zfill(2)+'_noPBC.dcd')

In [ ]:
def distance(index1, index2):
    
    atom1_xyz = traj.xyz[:,index1]
    atom2_xyz = traj.xyz[:,index2]

    d = np.linalg.norm(atom1_xyz - atom2_xyz, axis=1)
    
    return d

In [ ]:
atom1 = traj.topology.residue(0).atom('CA')
atom2 = traj.topology.residue(40).atom('CA')

In [ ]:
atom3 = traj.topology.residue(120).atom('CA')
atom4 = traj.topology.residue(42).atom('CA')

In [ ]:
plt.plot(distance(atom1.index, atom2.index))
plt.plot(distance(atom3.index, atom4.index))
plt.ylim(0,5)
plt.ylabel('Distance [Angstroms]')